# 1: EDA

Pandas has a `cut` method to create bins. 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()
data = pd.DataFrame(columns=diabetes['feature_names'], data = diabetes['data'])
s3s = pd.cut(data['s3'], 10) # assigns an interval to each index
s3s.value_counts().sort_index() # sort index sorts on start of interval here

Pyplot's histogram is actually this same information organized differently.

In [ ]:
import matplotlib.pyplot as plt
h = plt.hist(data['s3'], bins=10);
counts = [int(x) for x in h[0] ]
cutoffs = [round(x,3) for x in h[1]] # cutoffs
for i in range(len(h[0])):
    print(f'( {cutoffs[i]}, {cutoffs[i+1]} ]    {counts[i]}' )

Pandas supports smoothing of histograms to approx density (not normalized)

In [ ]:
# plt.density(data['s3']); #DNE
data['s3'].plot.density()


## Estimates of Location 

**Timmed mean** is the mean of the est with the highest 10% and lowest 10% removed... this is to reduce sensativity to outliers. Pandas doesn't have it, scipy.stats does.



In [ ]:
from scipy import stats

In [ ]:
stats.trim_mean(data['s3'], 
                proportiontocut= .1, # there is no default here
               ), data['s3'].mean() # to see that they are not the same

In [ ]:
# to check that this is what I expect it to be.. I'll come up with a 
# alternative with pandas quantile function:
low = data['s3'].quantile(.1)
high = data['s3'].quantile(.9)

fil = (data['s3']>=low) & (data['s3']<=high)

( data['s3'][ fil]).mean() # dang it... why is this not the same?

In [ ]:
low, high

## Estimates of Variablility

**Mean Absolute deviation from the median**
$\text{mean}\left\{ |x_i -m| \right\}$ where
$m = \text{median}\{x_i\}$. 

In [ ]:
m = data['s3'].median()
devs = data['s3'] - m
abdevs = devs.map(abs)
mad = abdevs.mean()
mad

**Median Absolute Deviation from the Median**
$\text{median}\left\{ |x_i -m| \right\}$ where
$m = \text{median}\{x_i\}$. 

In [ ]:
m = data['s3'].median()
devs = data['s3'] - m 
abdevs = devs.map(abs)
mead = abdevs.median()
mead

In [ ]:
plt.hist(data['s3'], alpha =0.5)
plt.vlines(x=[m-mad,m, m+mad], ymin=0, ymax =100, color='salmon', alpha = 0.9, label ='MAD')
plt.vlines(x=[m-mead,m, m+mead], ymin=0, ymax =90, color='red', alpha = .9, label= 'MeaAD')
plt.legend();

## Exploring 2 or more variables

When there are so many points to plot that it is visually overwhelming, it is nice to use either contour plots or hexbin.

In [ ]:
# note that this is a method from pandas plot library. 
data.plot.hexbin(x='s3', y='s2', 
                 gridsize=30,
                );

When you have two categorical variables, you might want a table of counts of things in two categories. That is a kind of pivot table. 

In [ ]:
df = pd.read_csv('data/train.csv')

c_feats = [ 'MS Zoning', 'Lot Shape'] 
dfc = df[c_feats]

dfc.pivot_table(index = 'MS Zoning', columns = 'Lot Shape', 
               aggfunc = lambda x : len(x),
               margins = True,
              )

## Correlation

There are other types of correlation coefficient, since Pearson's has some flaws. 

### Spearman's rho 
$\rho$

Given two lists of the same length of values of observables
- give the values of the first a ranking
- give the values of the second a ranking, 
- calculate the sum of the squares of the distances between rankings
- $\rho = 1-\frac{6\sum d^2}{n(n^2 -1)}$
- this is the pearson correlation between the two rank series. 
- Spearman's $\rho$ is thus a measure of monotonicity of $x$ vs $y$, regardless of the linearity. 
- I also not that the original order of the points is forgotten in Spearman, but not Pearson. 

In [ ]:
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()
df = pd.DataFrame(data = diabetes.data, columns= diabetes.feature_names)
df['s2'].rank()[:3] # there is a nice method to give the rank of each entry, with averaging :)

In [ ]:
df['dsq'] = (df['s2'].rank() - df['s3'].rank())**2
n = len(df)
1-6*df['dsq'].sum()/(n*(n+1)**2)

In [ ]:
stats.spearmanr(df['s2'],df['s3'])

In [ ]:
stats.pearsonr(df['s2'].rank(), df['s3'].rank())

### Kendal's tau 

Let $a_{ij}  = {\text{Sign}} (x_i-x_j)$ and Let $b_{ij} = \text{sign}(y_i-y_j)$. 

Then Kendal's $\tau = \frac{\langle a,b \rangle }{\lVert a\rVert \lVert b \rVert}$. 

This is the normalized count of concordant pairs: $(x_1,y_y), ~(x_2,y_2)$ such that $the order of $x_1,x_2$ is the same as the order of $y_1,y_2$.

There is a generalized form of this for any antisymmetric matrix $a$.

## Categorical and Numeric Data (tegether)

In [ ]:
# pandas has a nice way to show box plots of a quantity for each value of a categorical vbariable. 
df.boxplot(by=c_feats[0], column = 'Lot Area');

In [ ]:
import bokeh # looks badass for plotting


# 2: Data and Sampling Distributions

**The vast search effect**

If you torture a dataset long enough it will eventually confess... but that confession is not trustworthy. Running lots of models and asking lots of question is not always good. 

**Target Shuffling** is a data science approach to significance testing: you found a good model, lovely. Record its $R2$ score (as an example) and then
- shuffle the target data
- run the fit
- record whether you got a $R^2$ score better than that or not. 
- repeat 100, or 100,000 times. 
- What fraction of the times do you get a better result? 5%? Then there is a 5% chance that your good model came from chance alone. 

**Samipling error** is the term for standard deviation of a sampling distribution. 

## Bootsraping
is the modern way to estimate confidence intervals. 

Note that the most intuitive interpretation of a confidence interval "95% probability that the true value lies within..." is wrong. THe correct wording must start with "given a sampling procedure and a population, what is the probability that". 

I believe the correct interpretation is "if we sampled this population 100 times, 95% of the time we'd get intervals that include the population mean 95% of the time. 

## Prediction Intervals vs CIs

PIs 
- are about where you expect future values to fall
- aren't where you expect the population mean to be. 
- are always bigger than CIs since ngs must be taken into account:
    1. The variation in estimating the population mean from a sample (the standard error)
    2. The variation in the observation around the mean. 
    


Following <a href="https://towardsdatascience.com/confidence-intervals-vs-prediction-intervals-7b296ae58745">Michal</a>
- calculate PI and CI using boostrapping. 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression

housing = fetch_california_housing()
df = pd.DataFrame(data = housing['data'], columns = housing['feature_names'])
df['target'] = housing['target'] 
df = df[['MedInc','target']].sample(200) # two columns, 200 rows

# df.head(3)

lr = LinearRegression(fit_intercept=True)

mu_dist = list()
pred_dist = list()

for _ in range(1_000):
    sample = df.sample(len(df), replace=True)
    lr.fit(sample[['MedInc']], sample['target'])
    
    pred = lr.predict(pd.DataFrame({'MedInc':[3]}))[0] #prediction for 3

    # calculate residuals
    preds = lr.predict(sample[['MedInc']][:1]) 
    residuals =  (sample['target'] - preds)

    mu_dist.append(pred)
    pred_dist.append(pred+ residuals.sample(1))
                                               
print(f'{round(np.mean(mu_dist),2)}, CI = {np.percentile(mu_dist, [5,95] )} ')
print(f'{round(np.mean(pred_dist),2)}, PI = { np.percentile(pred_dist, [5,95])} ')
      

# Q-Q plot 
**Handmade:**

In [ ]:
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt

Experiment: Flip a coin 2,000 times. 

Wanted: 1000 observations of the number of heads in 2,000 flips.  

i.e. 1,000 samples from a n=2000 binomial distributionn with P=.53. 

In [ ]:
n = 2000
samples = np.random.binomial(n, 0.53, size = 1000)
# samples.shape # (1000,)
# samples[:20] #each of these is the sum of a sample, ~ of size 0.53*2000 = 1033
# lets look at the p values of the samples, they should be p close to P=0.53
ps = samples/n
# ps[:20] # yup

# standardize the observation
zs = (ps-np.mean(ps))/np.std(ps)
zs[:20]

In [ ]:
# how do I get quantiles of a list? np has quantile
xs = np.linspace(0,1,100)
qs = np.quantile(zs,xs) # quantiles of my distribution
plt.scatter(stats.norm().ppf(xs), qs, label= 'QQ',alpha=0.5) 
plt.plot(stats.norm().ppf(xs),stats.norm().ppf(xs), color='salmon',label='line')
plt.xlabel('Normal')
plt.title("Handmade QQ Plot")
plt.legend();

**Implementation using scipy.stats**

In [ ]:
stats.probplot(qs, #plot the quantiles of these standardized means of samples
               dist="norm", # norm is default, this sets the x axis 
               plot=plt) # show the plot
plt.title("Normal Q-Q plot");
# the binning seems to be quite different than what I chose. 
# plt.show()

**Implementation using statsmodels**

In [ ]:
import numpy as np 
import statsmodels.api as sm 
import pylab as py 
  

In [ ]:
# np.random generates different random numbers everytime the code is executed.
data_points = np.random.normal(0, 1, 100)     
  
sm.qqplot(data = zs, # data_points, 
          # dist = idk how to tell it, but normal is default
          line ='45',
         ) 
py.show();

This looks off

what the heck yo? 


## Poisson vs Exponential

The Poisson disribution is discrete and interepreted as the number of events per unit time when there is an expected number of events pre unit time. 

In [ ]:
from scipy import stats

In [ ]:
X = stats.poisson(mu = 2)

In [ ]:
X.rvs(size = 20)

By contrast, the exponential distribution is continuous and give the time between events when the number of events per unit time is known. 

In [ ]:
Y = stats.expon(scale = 2)
Y.rvs(size = 20)

## Weibill Distribution

has a very nice CDF 
$$
F(x) = 1-e^{-\left(\frac{x}{\eta} \right)^a}
$$

and is very similar to the logistic CDF (by virtueo f being the Taylor approx?)
$$
F(x) = \frac{1}{1+e^{-a(x-\eta) }}
$$

In [ ]:
from scipy import stats

a,loc = 3, 5
X = stats.weibull_min(c=a, loc=loc)
Y = stats.logistic(loc=loc+1, scale=1/a)

xs = np.linspace(0,10,100)
ysl = Y.cdf(xs)
ysw = X.cdf(xs)

plt.plot(xs,ysw, label='Weibel')
plt.plot(xs,ysl, label = 'Logistic')
plt.legend();

Wind speed distributions are best modeled as Weibill distributions, and are less accurately approximated by Rayleigh (the $a=2$ case.)

idk what the max vs min is about. Apparently the dist is for max and min of iid rand vars.
<a href = 'https://github.com/scipy/scipy/issues/10014'>see</a>.

In [ ]:
# Weibill max = - Weibill min
M = stats.weibull_max(c=a, loc=loc)
ysm = M.cdf(xs)
plt.plot(xs,ysm, alpha =0.5)
plt.plot(xs,ysw, alpha = .5);

## The Gumbel dsitribution
is nice for modeling time at failure of things that tend to die at a certain time.
$$
f(T) =\frac1\sigma e^{-e^{(T-\mu)/\sigma}}
$$

In [ ]:
Z = stats.gumbel_r(loc=loc, scale =1/a)
ysg = Z.cdf(xs)
plt.plot(xs,ysg);

# ch 3 Statistical Experiments`